In [1]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json

class JsonModelParser:
    def __init__(self, filepath):
        self.filepath = filepath
        self.data = self.load_json_file()
        self.parse_model_data()

    def printDetails(self):
        print("\tINPUT SIZE:",self.input_size)
        print("\tLAYERS:", self.layers)
        print("\tINPUT:", self.input)
        print("\tINITIAL WEIGHTS:", self.initial_weights)
        print("\tTARGET:", self.target)
        print("\tLEARNING RATE:", self.learning_rate)
        print("\tBATCH SIZE:", self.batch_size)
        print("\tMAX ITERATION:", self.max_iteration)
        print("\tERROR THRESHOLD:", self.error_threshold)
    

    def load_json_file(self):
        try:
            with open(self.filepath, 'r', encoding='utf-8') as file:
                return json.load(file)
        except FileNotFoundError:
            print(f"The file {self.filepath} was not found")
            return None
        except json.JSONDecodeError:
            print(f"Error decoding JSON from the file {self.filepath}")
            return None

    def parse_model_data(self):
        if self.data:
            self.case = self.data.get('case', {})
            self.model = self.case.get('model', {})
            self.input_size = self.model.get('input_size')

            raw_layers = self.model.get('layers', [])   
            self.layers = [{'number_of_neurons': layer.get('number_of_neurons'),
                        'activation_function': layer.get('activation_function')}
                       for layer in raw_layers]
            
            self.input = self.case.get('input', [])
            print(self.case.get("initial_weights"))
            self.initial_weights = self.case.get('initial_weights', [])
            self.target = self.case.get('target', [])
            self.parameters = self.case.get('learning_parameters', {})

            self.learning_rate = self.parameters.get('learning_rate')
            self.batch_size = self.parameters.get('batch_size')
            self.max_iteration = self.parameters.get('max_iteration')
            self.error_threshold = self.parameters.get('error_threshold')

            self.expect = self.data.get('expect', {})
            self.stopped_by = self.expect.get('stopped_by', '')
            self.final_weights = self.expect.get('final_weights', [])

    @staticmethod
    def save_json_file(data, filepath):
        try:
            with open(filepath, 'w', encoding='utf-8') as file:
                json.dump(data, file, ensure_ascii=False, indent=4)
        except IOError:
            print(f"Could not save data to {filepath}")

In [3]:
import tensorflow as tf
import numpy as np

class TensorFlowModel: 
    def __init__(self, inputs, targets, learning_rate, batch_size, initial_weights, layers) -> None:
        self.inputs = inputs
        self.targets = targets
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.input_size = len(inputs[0])
        self.initial_weights = initial_weights
        self.dataset = tf.data.Dataset.from_tensor_slices((self.inputs, self.targets))
        self.dataset = self.dataset.batch(batch_size)
        self.model = tf.keras.Sequential()
        
        for i, layer in enumerate(layers):
            if i == 0:
                self.model.add(tf.keras.layers.Dense(
                    layer['number_of_neurons'],
                    activation=layer['activation_function'],
                    input_shape=(self.input_size,),
                    kernel_initializer=lambda shape, dtype: tf.constant_initializer(initial_weights[i][1:])(shape, dtype=dtype),
                    bias_initializer=lambda shape, dtype: tf.constant_initializer(initial_weights[i][0])(shape, dtype=dtype)
                ))
            else:
                self.model.add(tf.keras.layers.Dense(
                    layer['number_of_neurons'],
                    activation=layer['activation_function'],
                    kernel_initializer=lambda shape, dtype: tf.constant_initializer(initial_weights[i][1:])(shape, dtype=dtype),
                    bias_initializer=lambda shape, dtype: tf.constant_initializer(initial_weights[i][0])(shape, dtype=dtype)
                ))
        
        loss_function = tf.keras.losses.CategoricalCrossentropy() if layers[-1]['activation_function'] == 'softmax' else tf.keras.losses.MeanSquaredError()
        optimizer = tf.keras.optimizers.SGD(learning_rate=self.learning_rate)
        self.model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

    def fit_model(self, max_epochs, error_threshold):
        class ThresholdCallback(tf.keras.callbacks.Callback):
            def on_epoch_end(self, _, logs = None):
                if(logs.get("loss") < error_threshold):
                    self.model.stop_training = True
        thressholdCallback = ThresholdCallback()
        self.model.fit(self.dataset, epochs=max_epochs, callbacks=[thressholdCallback])
    def predict(self):
        input_array = np.array(self.inputs)
        if input_array.ndim == 1:
            input_array = np.expand_dims(input_array, axis=0)
        
        results = self.model.predict(input_array)
        return results
    
    def show_prediction(self):
        total_weights = []
        total_biases = []
        print("============================= Result =============================\n")
        if(self.model.stop_training):
            print(f"Stopped by : error_threshold\n")
        else:
            print("Stopped by : max_iteration\n")
        for i,layer in enumerate(self.model.layers):
            print(layer.get_weights())
            weights, biases = layer.get_weights()
            total_weights.append(weights)
            total_biases.append(biases)
            print(f"Layer-{i}")
            print(f"Weights : {weights}\n")
            print(f"Biases : {biases}\n")
        print("==================================================================")
        return total_weights, total_biases


In [ ]:
import os
import csv

def main():
    currDir = os.getcwd()
    testDir = currDir.replace("src", "test")
    file = os.path.join(testDir, "softmax_two_layer.json")
    json_file = JsonModelParser(file)
    inputs = json_file.input
    targets = json_file.target
    learning_rate = json_file.learning_rate
    layers = json_file.layers
    batch_size = json_file.batch_size
    initial_weights = json_file.initial_weights
    max_iter = json_file.max_iteration
    error_threshold = json_file.error_threshold
    model = TensorFlowModel(inputs, targets, learning_rate, batch_size, initial_weights, layers)
    model.fit_model(max_iter, error_threshold)
    weights, biases = model.show_prediction()

main()

[[[0.1, -0.1, 0.1, -0.1], [-0.1, 0.1, -0.1, 0.1], [0.1, 0.1, -0.1, -0.1]], [[0.12, -0.1], [-0.12, 0.1], [0.12, -0.1], [-0.12, 0.1], [0.02, 0.0]]]
Ini json_file :  <__main__.JsonModelParser object at 0x00000186B9FED210>
ini inputs :  [[3.99, 2.96], [-0.71, 2.8], [-2.43, -0.2], [-1.9, 2.62], [-2.58, 1.43], [-3.43, -0.25], [1.15, -2.3], [4.28, 3.45]]
Epoch 1/200


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3328 - loss: 0.7636      
Epoch 2/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4450 - loss: 0.7224    
Epoch 3/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5505 - loss: 0.7109    
Epoch 4/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6127 - loss: 0.7006    
Epoch 5/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5283 - loss: 0.6914    
Epoch 6/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6127 - loss: 0.6799    
Epoch 7/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6127 - loss: 0.6608    
Epoch 8/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6127 - loss: 0.6347    
Epoch 9/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6127 - loss: 0.5831    
Epoch 10/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6127 - loss: 0.5502    
Epoch 11/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9722 - loss: 0.5073
Epoch 12/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc

In [ ]:
# IRIS

inputs = []
targets = []
all_species = []
with open("iris.csv", "r") as csvfile:
    # Gunakan csv.DictReader
    datareader = csv.DictReader(csvfile)

    # Tidak perlu next(datareader), DictReader sudah meng-handle header
    for row in datareader:
        # Sekarang row adalah dictionary, dan Anda bisa mengakses nilai menggunakan nama kolom
        current = [
            float(row["SepalLengthCm"]),
            float(row["SepalWidthCm"]),
            float(row["PetalLengthCm"]),
            float(row["PetalWidthCm"])
        ]
        species = row["Species"]
        all_species.append(species)
        if(species == "Iris-setosa"):
            target = [1,0,0]
        elif(species == "Iris-versicolor"):
            target = [0,1,0]
        elif(species == "Iris-virginica"):
            target = [0,0,1]
        inputs.append(current)
        targets.append(target)

In [ ]:
architecture = JsonModelParser("iris_config.json")
architecture.input_size = len(inputs[0])
architecture.input = inputs
architecture.target = targets
model = TensorFlowModel(inputs, targets, architecture.learning_rate, architecture.batch_size, architecture.initial_weights, architecture.layers)
model.fit_model(architecture.max_iteration, architecture.error_threshold)
model.show_prediction()

[[[0.1, 0.1, 0.1], [0.1, 0.1, 0.1], [0.1, 0.1, 0.1], [0.1, 0.1, 0.1], [0.1, 0.1, 0.1]]]
Epoch 1/100


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7676 - loss: 1.0154 
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3302 - loss: 1.1021  
Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2872 - loss: 1.1125  
Epoch 4/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3167 - loss: 1.0888  
Epoch 5/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3446 - loss: 1.0538  
Epoch 6/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4953 - loss: 1.0165   
Epoch 7/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5521 - loss: 0.9801   
Epoch 8/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7515 - loss: 0.9455
Epoch 9/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8060 - loss: 0.9132
Epoch 10/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8079 - loss: 0.8831
Epoch 11/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8079 - loss: 0.8550
Epoch 12/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accura

([array([[ 0.32361224,  0.1785121 , -0.20212464],
         [ 0.7047748 , -0.0998018 , -0.30497408],
         [-0.7898869 ,  0.2469572 ,  0.8429315 ],
         [-0.30710194,  0.03382678,  0.57327604]], dtype=float32)],
 [array([ 0.22143798,  0.133197  , -0.05463498], dtype=float32)])

In [ ]:
import numpy as np
results = model.predict()
res =[]
for i in range(len(results)):
    idmax = np.argmax(results[i])
    if idmax == 0:
        res.append('Iris-setosa')
    elif idmax == 1:
        res.append('Iris-versicolor')
    else:
        res.append('Iris-virginica')
print(res)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virgin

In [ ]:
false_count = 0
for i in range(len(res)):
    if(res[i] != all_species[i]):
        print(f"Res ke - {i} : {res[i]}\n Species ke - {i} : {all_species[i]}")
        false_count += 1

false_count

Res ke - 50 : Iris-virginica
 Species ke - 50 : Iris-versicolor
Res ke - 51 : Iris-virginica
 Species ke - 51 : Iris-versicolor
Res ke - 52 : Iris-virginica
 Species ke - 52 : Iris-versicolor
Res ke - 53 : Iris-virginica
 Species ke - 53 : Iris-versicolor
Res ke - 54 : Iris-virginica
 Species ke - 54 : Iris-versicolor
Res ke - 55 : Iris-virginica
 Species ke - 55 : Iris-versicolor
Res ke - 56 : Iris-virginica
 Species ke - 56 : Iris-versicolor
Res ke - 58 : Iris-virginica
 Species ke - 58 : Iris-versicolor
Res ke - 59 : Iris-virginica
 Species ke - 59 : Iris-versicolor
Res ke - 60 : Iris-virginica
 Species ke - 60 : Iris-versicolor
Res ke - 61 : Iris-virginica
 Species ke - 61 : Iris-versicolor
Res ke - 62 : Iris-virginica
 Species ke - 62 : Iris-versicolor
Res ke - 63 : Iris-virginica
 Species ke - 63 : Iris-versicolor
Res ke - 65 : Iris-virginica
 Species ke - 65 : Iris-versicolor
Res ke - 66 : Iris-virginica
 Species ke - 66 : Iris-versicolor
Res ke - 67 : Iris-virginica
 Species ke

44